# À propos de ce livret

Recherches liées au développement de cette méthode : 
>Mehmet Gonen, Samuel Kaski (2014). **Kernelized Bayesian Matrix Factorization**.  
>Mehmet Gonen, Suleiman A. Khan, Samuel Kaski (2013). **Kernelized Bayesian Matrix Factorization**.


La méthode implantés provient de ces sources : 
>[**Dépôt 1**](https://github.com/mehmetgonen/kbmf) | *Version en R et MATLAB*



**Note sur le cachier**
- Revoir la décomposition (génération des facteurs latents pour la création des noyaux)
- Améliorer la performance du code (Rcpp ?)

## Préparation pour l'utilisation

In [2]:
# Chargement des données
library(R.matlab)

# Barre de progression
library(progress)

R.matlab v3.6.2 (2018-09-26) successfully loaded. See ?R.matlab for help.


Attaching package: 'R.matlab'


The following objects are masked from 'package:base':

    getOption, isOpen




In [3]:
setwd("C:/Users/amass/OneDrive/02_Education/02_Maitrise/Cours/PROJET_MAITRISE/application")

# Données

## Chargement des données

In [4]:
# Tenseur de données
tensor <- readMat("data/Guangzhou-data-set/tensor.mat")
tensor <- tensor$tensor  # Prendre l'array 
dim(tensor)

# Matrice de données aléatoires
random_matrix <- readMat("data/Guangzhou-data-set/random_matrix.mat")
random_matrix <- random_matrix$random.matrix
dim(random_matrix)

# Tenseur de données aléatoires
random_tensor <- readMat("data/Guangzhou-data-set/random_tensor.mat")
random_tensor <- random_tensor$random.tensor
dim(random_tensor)

[1] 214  61 144

[1] 214  61

[1] 214  61 144

## Format des données

In [5]:
# Indiquer si utiliser le jeu de données tronqué ou complet
petit_donnees = TRUE
# Indiquer si je veux travailler avec un tenseur ou une matrice
matrice = TRUE

capteurs = 50
jours = 10
sequences = dim(tensor)[3]  # (la séquence de temps du tenseur est pleine)

# Choix de données tronquées ou complètes
if (petit_donnees == TRUE) {
    # Si l'utilisateur choisi de travailler avec les données tronquées
    tensor = tensor[1:capteurs, 1:jours, 1:sequences, drop = FALSE]
    
    random_tensor = random_tensor[1:capteurs, 1:jours, 1:sequences, drop = FALSE]
}

# Choix de matrice ou de tenseur
if (matrice == TRUE) {
    # Si l'utilisateur choisi de travailler avec une matrice
    mat_dense <- array(tensor, c(dim(tensor[, , ])[1], dim(tensor[, , ])[2] * dim(tensor[, 
        , ])[3]))
}

dim(tensor)
dim(mat_dense)

[1]  50  10 144

[1]   50 1440

## Scénarios de manquants

In [6]:
tx_manquant = 0.2
manquants_aleatoires = TRUE

# Manquants aléatoires
if (manquants_aleatoires == TRUE) {
    print("Manquants aléatoires")
    mat_binaire <- round(random_tensor + 0.5 - tx_manquant)
    mat_binaire <- array(mat_binaire, c(dim(random_tensor[, , ])[1], dim(random_tensor[, 
        , ])[2] * dim(random_tensor[, , ])[3]))
    dim(mat_binaire)
    
    # Création de la matrice de données manquantes (element-wise)
    mat_manq <- mat_dense * mat_binaire
    head(mat_manq)
}

# Manquants non-aléatoires
if (manquants_aleatoires == FALSE) {
    print("Manquants non-aléatoires")
    tens_binaire <- array(0, dim(tensor))
    for (i1 in 1:dim(tensor)[1]) {
        for (i2 in 1:dim(tensor)[2]) {
            tens_binaire[i1, i2, ] <- round(random_matrix[i1, i2] + 0.5 - tx_manquant)
        }
    }
    mat_binaire <- array(tens_binaire, c(dim(tens_binaire)[1], dim(tens_binaire)[2] * 
        dim(tens_binaire)[3]))
    dim(mat_binaire)
    
    # Création de la matrice de données manquants (element-wise)
    mat_manq <- mat_dense * mat_binaire
    head(mat_manq)
}

[1] "Manquants aléatoires"


40.893,41.227,42.680,0.000,42.804,40.020,0.000,35.357,41.097,41.821,...,0.000,40.870,0.000,42.295,39.082,0.000,39.008,0.000,39.974,0.000
50.319,0.000,47.984,50.660,51.622,50.770,50.542,50.463,52.192,0.000,...,51.534,0.000,51.192,50.546,50.227,50.693,51.067,0.000,51.232,0.000
0.000,57.001,49.905,52.292,55.792,52.594,50.130,0.000,0.000,55.341,...,53.983,50.571,52.489,55.382,54.422,0.000,52.618,0.000,53.334,52.322
37.305,0.000,37.195,0.000,0.000,33.248,31.593,38.236,34.700,0.000,...,36.883,0.000,37.362,34.821,0.000,32.957,0.000,34.647,34.758,37.017
38.388,39.534,35.762,36.393,36.625,37.164,34.079,0.000,37.764,0.000,...,37.508,35.285,37.175,37.707,36.642,34.773,37.572,36.967,37.361,36.306
0.000,44.112,43.173,0.000,48.262,47.250,46.281,48.180,0.000,49.987,...,51.562,45.329,50.494,48.465,46.354,45.446,49.026,45.500,47.928,47.412


## Création des noyaux


>Kevin P. Murphy (2012), **Machine Learning : A Probabilistic Perspective.**

In [8]:
set.seed(1606)

Px <- 1 # isotropic dimensions
Nx <- dim(mat_dense)[2] # X
Pz <- 1 # isotropic dimensions
Nz <- dim(mat_dense)[1] # Z

X <- matrix(rnorm(Px * Nx), Px, Nx)
Z <- matrix(rnorm(Pz * Nz), Pz, Nz)
Y <- sparse_mat

In [10]:
# Création des noyaux

## X
Kx <- array(0, c(Nx, Nx, Px))
for (m in 1:Px) {
  Kx[, , m] <- crossprod(X[m, , drop = FALSE], X[m, , drop = FALSE])
}

## Z
Kz <- array(0, c(Nz, Nz, Pz))
for (n in 1:Pz) {
  Kz[, , n] <- crossprod(Z[n, , drop = FALSE], Z[n, , drop = FALSE])
}

, , 1

                 [,1]          [,2]         [,3]          [,4]         [,5]
   [1,]  7.730510e-04  0.0157555438 -0.028198414  0.0092632326 -0.026150204
   [2,]  1.575554e-02  0.3211135617 -0.574711556  0.1887938390 -0.532967018
   [3,] -2.819841e-02 -0.5747115556  1.028587427 -0.3378929259  0.953875327
   [4,]  9.263233e-03  0.1887938390 -0.337892926  0.1109984688 -0.313349859
   [5,] -2.615020e-02 -0.5329670175  0.953875327 -0.3133498591  0.884589988
   [6,]  5.302670e-02  1.0807364891 -1.934243280  0.6354025962 -1.793748293
   [7,] -3.262577e-02 -0.6649454524  1.190083138 -0.3909445744  1.103640695
   [8,] -2.202096e-02 -0.4488089103  0.803253732 -0.2638703788  0.744908889
   [9,]  5.650020e-02  1.1515300030 -2.060945654  0.6770245670 -1.911247560
  [10,]  3.120598e-02  0.6360087769 -1.138293853  0.3739316957 -1.055613158
  [11,]  2.485476e-02  0.5065646554 -0.906621818  0.2978269915 -0.840768768
  [12,] -2.097035e-02 -0.4273964179  0.764930820 -0.2512812293  0.709369585
  [13,]  3.165523e-02  0.6451648452 -1.154680885  0.3793148670 -1.070809908
  [14,] -3.368673e-02 -0.6865688700  1.228783553 -0.4036577342  1.139530081
  [15,] -8.155721e-03 -0.1662216584  0.297494467 -0.0977274982  0.275885768
  [16,]  2.923554e-02  0.5958492062 -1.066418442  0.3503204864 -0.988958463
  [17,]  8.555732e-03  0.1743742715 -0.312085570  0.1025207032 -0.289417036
  [18,] -1.358259e-03 -0.0276826546  0.049544907 -0.0162755961  0.045946181
  [19,]  4.800460e-02  0.9783811522 -1.751053275  0.5752243312 -1.623864411
  [20,] -1.366910e-02 -0.2785898040  0.498604851 -0.1637926419  0.462388372
  [21,] -7.366569e-03 -0.1501379529  0.268708727 -0.0882713279  0.249190898
  [22,] -1.163955e-02 -0.2372255452  0.424573355 -0.1394731545  0.393734200
  [23,]  1.634482e-02  0.3331235174 -0.596206320  0.1958549100 -0.552900496
  [24,]  3.571056e-02  0.7278165000 -1.302606314  0.4279086515 -1.207990679
  [25,]  2.737819e-02  0.5579944831 -0.998668121  0.3280643772 -0.926129229
  [26,] -6.874227e-04 -0.0140103552  0.025074970 -0.0082371754  0.023253634
  [27,] -4.625182e-02 -0.9426577529  1.687117482 -0.5542213014  1.564572634
  [28,]  2.704973e-02  0.5513002388 -0.986687128  0.3241285980 -0.915018483
  [29,] -2.411657e-02 -0.4915195364  0.879694884 -0.2889814424  0.815797688
  [30,]  4.122082e-02  0.8401211216 -1.503603006  0.4939364472 -1.394387849
  [31,]  3.993206e-02  0.8138548719 -1.456593104  0.4784936048 -1.350792540
  [32,]  1.368759e-02  0.2789667123 -0.499279421  0.1640142394 -0.463013944
  [33,] -3.547250e-02 -0.7229646960  1.293922819 -0.4250561070  1.199937915
  [34,]  5.791870e-03  0.1180440279 -0.211268769  0.0694021925 -0.195923128
  [35,] -1.880758e-02 -0.3833171728  0.686040189 -0.2253655070  0.636209225
  [36,] -4.657902e-02 -0.9493265616  1.699052952 -0.5581421262  1.575641164
  [37,]  1.263120e-02  0.2574364155 -0.460745669  0.1513558286 -0.427279115
  [38,]  2.083218e-02  0.4245803901 -0.759890847  0.2496255885 -0.704695694
  [39,]  2.429420e-02  0.4951398772 -0.886174372  0.2911099668 -0.821806535
  [40,]  5.726621e-02  1.1671419939 -2.088887144  0.6862033998 -1.937159503
  [41,] -2.661633e-02 -0.5424672077  0.970878250 -0.3189353516  0.900357893
  [42,] -7.923752e-03 -0.1614938962  0.289032976 -0.0949478823  0.268038882
  [43,]  4.885958e-03  0.0995806634 -0.178224045  0.0585469378 -0.165278629
  [44,]  1.910239e-02  0.3893256483 -0.696793831  0.2288980988 -0.646181770
  [45,]  1.530951e-02  0.3120229233 -0.558441626  0.1834491364 -0.517878865
  [46,] -4.029269e-02 -0.8212048466  1.469747688 -0.4828149107  1.362991633
  [47,] -1.056381e-02 -0.2153009353  0.385333883 -0.1265829133  0.357344912
  [48,]  2.241881e-02  0.4569175580 -0.817766148  0.2686377350 -0.758367186
  [49,]  9.849943e-04  0.0200751580 -0.035929424  0.0118028841 -0.033319667
  [50,] -7.412424e-02 -1.5107253232  2.703813865 -0.8882079972  2.507420632
  [51,]  2.585957e-03  0.0527043591 -0.094327390  0.0309867270 -0.087475860
  [52,]  2.890107e-02  0.5890322901 -1.05

, , 1

             [,1]         [,2]        [,3]         [,4]        [,5]        [,6]
 [1,]  1.51348591 -0.160200502  0.59170062 -0.094443890 -1.34439962 -0.95159201
 [2,] -0.16020050  0.016957013 -0.06263074  0.009996762  0.14230294  0.10072477
 [3,]  0.59170062 -0.062630736  0.23132665 -0.036923045 -0.52559597 -0.37202697
 [4,] -0.09444389  0.009996762 -0.03692304  0.005893447  0.08389264  0.05938083
 [5,] -1.34439962  0.142302939 -0.52559597  0.083892641  1.19420361  0.84528038
 [6,] -0.95159201  0.100724768 -0.37202697  0.059380831  0.84528038  0.59830577
 [7,]  0.73190275 -0.077470947  0.28613898 -0.045671877 -0.65013474 -0.46017792
 [8,] -0.38288355  0.040527723 -0.14968916  0.023892533  0.34010789  0.24073493
 [9,] -2.13523807  0.226012153 -0.83477598  0.133242198  1.89668977  1.34251364
[10,] -1.92999020  0.204286935 -0.75453388  0.120434410  1.71437215  1.21346571
[11,] -0.65720997  0.069564815 -0.25693767  0.041010931  0.58378663  0.41321545
[12,] -1.35824863  0.143768839 -0.53101027  0.084756841  1.20650541  0.85398783
[13,]  0.39207054 -0.041500153  0.15328083 -0.024465815 -0.34826851 -0.24651117
[14,] -1.75202891  0.185449966 -0.68495953  0.109329347  1.55629265  1.10157399
[15,] -0.21909906  0.023191349 -0.08565726  0.013672125  0.19462137  0.13775676
[16,]  0.90995870 -0.096317936  0.35575034 -0.056782847 -0.80829832 -0.57212916
[17,] -0.09017868  0.009545296 -0.03525555  0.005627291  0.08010394  0.05669912
[18,]  1.25614206 -0.132960992  0.49109148 -0.078385231 -1.11580616 -0.78978915
[19,]  0.19562328 -0.020706468  0.07647935 -0.012207199 -0.17376829 -0.12299655
[20,] -0.07796177  0.008252151 -0.03047932  0.004864936  0.06925190  0.04901783
[21,]  2.70465050 -0.286283713  1.05738901 -0.168774425 -2.40248758 -1.70052709
[22,] -0.12466099  0.013195202 -0.04873648  0.007779041  0.11073389  0.07837959
[23,] -0.54820430  0.058026708 -0.21432167  0.034208807  0.48695905  0.34467902
[24,]  1.30347994 -0.137971645  0.50959833 -0.081339189 -1.15785547 -0.81955246
[25,]  0.09316865 -0.009861779  0.03642449 -0.005813869 -0.08275987 -0.05857903
[26,] -0.32683732  0.034595303 -0.12777777  0.020395161  0.29032314  0.20549632
[27,] -1.06920457  0.113173903 -0.41800786  0.066720039  0.94975328  0.67225371
[28,] -0.56272357  0.059563553 -0.21999801  0.035114832  0.49985622  0.35380789
[29,] -0.20215355  0.021397689 -0.07903237  0.012614698  0.17956901  0.12710241
[30,] -1.44477843  0.152927904 -0.56483928  0.090156435  1.28336812  0.90839273
[31,]  0.20617131 -0.021822963  0.08060312 -0.012865413 -0.18313790 -0.12962854
[32,]  0.63228603 -0.066926648  0.24719360 -0.039455638 -0.56164718 -0.39754472
[33,]  1.34256458 -0.142108703  0.52487855 -0.083778131 -1.19257358 -0.84412661
[34,] -0.21920666  0.023202738 -0.08569932  0.013678839  0.19471695  0.13782441
[35,]  2.09680357 -0.221943912  0.81974993 -0.130843825 -1.86254917 -1.31834826
[36,] -0.25349077  0.026831666 -0.09910277  0.015818221  0.22517084  0.15938027
[37,]  0.41314047 -0.043730378  0.16151817 -0.025780612 -0.36698452 -0.25975873
[38,]  0.17870707 -0.018915909  0.06986592 -0.011151601 -0.15874196 -0.11236062
[39,] -0.35012740  0.037060527 -0.13688307  0.021848498  0.31101125  0.22013977
[40,] -1.37430879  0.145468786 -0.53728902  0.085759020  1.22077133  0.86408552
[41,] -0.85723057  0.090736734 -0.33513616  0.053492530  0.76146097  0.53897678
[42,] -1.48602294  0.157293582 -0.58096391  0.092730157  1.32000480  0.93432489
[43,] -0.96024793  0.101640986 -0.37541102  0.059920974  0.85296926  0.60374811
[44,] -2.08231054  0.220409843 -0.81408385  0.129939437  1.84967529  1.30923589
[45,] -1.85711316  0.196572996 -0.72604244  0.115886768  1.64963692  1.16764486
[46,]  0.27123635 -0.028710012  0.10604044 -0.016925573 -0.24093389 -0.17053766
[47,]  0.54145942 -0.057312770  0.21168474 -0.033787915 -0.48096770 -0.34043822
[48,]  1.23880131 -0.131125497  0.48431208 -0.077303141 -1.10040272 -0.77888629
[49,]  1.31863725 -0.139576026  0.51552411 -0.082285029 -1.17131941 -0.82

# Méthode

## Noyaux

**Réduction non linéaire de la dimensionalité par « noyaux »**

$({G_{x,m} = {A_x^T}{K_{x,m}}})_{m=1}^{P_x}$





*Dans la recherche : In order to calculate the similarity between drugs, we use a Gaussian kernel whose width parameter is selected as the square root of the dimensionality of the data points*

# Modélisation

- Quatre scénarios peuvent se présenter puisque les fonctions noyaux multiples peuvent être placés soit sur U, soit sur V ou sur les deux en même temps. Des stratégies différentes s'appliquent selon les applications de noyaux.
- À ces quatre scénarios s'ajoutent les stratégies pour les matrices complètes/ avec des valeurs manquantes.



## Un noyau sur les deux facteurs latents U et V

In [ ]:
kbmf1k1k_supervised_regression_variational_train <- function(Kx, Kz, Y, parameters) {
    set.seed(parameters$seed)
    # initialisation des paramètres
    Dx <- dim(Kx)[1]
    Nx <- dim(Kx)[2]
    
    Dz <- dim(Kz)[1]
    Nz <- dim(Kz)[2]
    
    R <- parameters$R
    
    sigma_g <- parameters$sigma_g
    sigma_y <- parameters$sigma_y
    
    # Lambda_x (initialisation du Lambda)
    Lambdax <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dx, R), beta = matrix(parameters$beta_lambda, 
        Dx, R))
    # A_x|Lambda_x
    Ax <- list(mu = matrix(rnorm(Dx * R), Dx, R), sigma = array(diag(1, Dx, Dx), 
        c(Dx, Dx, R)))
    # G_x|Lambda_x
    Gx <- list(mu = matrix(rnorm(R * Nx), R, Nx), sigma = diag(1, R, R))
    
    # Lambda_z (Initialisation du Lambda)
    Lambdaz <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dz, R), beta = matrix(parameters$beta_lambda, 
        Dz, R))
    # A_z|Lambda_z
    Az <- list(mu = matrix(rnorm(Dz * R), Dz, R), sigma = array(diag(1, Dz, Dz), 
        c(Dz, Dz, R)))
    # G_z|Lambda_z
    Gz <- list(mu = matrix(rnorm(R * Nz), R, Nz), sigma = diag(1, R, R))
    
    # Outer product of kernels
    KxKx <- tcrossprod(Kx, Kx)
    KzKz <- tcrossprod(Kz, Kz)
    
    # Variational algorithm
    for (iter in 1:parameters$iteration) {
        # update Lambdax
        for (s in 1:R) {
            Lambdax$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Ax$mu[, s]^2 + 
                diag(Ax$sigma[, , s])))
        }
        # update Ax
        for (s in 1:R) {
            Ax$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdax$alpha[, s] * 
                Lambdax$beta[, s]), Dx, Dx) + KxKx/sigma_g^2))
            Ax$mu[, s] <- Ax$sigma[, , s] %*% (tcrossprod(Kx, Gx$mu[s, , drop = FALSE])/sigma_g^2)
        }
        # update Gx
        Gx$sigma <- chol2inv(chol(diag(1/sigma_g^2, R, R) + (tcrossprod(Gz$mu, Gz$mu) + 
            Nz * Gz$sigma)/sigma_y^2))
        Gx$mu <- Gx$sigma %*% (crossprod(Ax$mu, Kx)/sigma_g^2 + tcrossprod(Gz$mu, 
            Y)/sigma_y^2)
        
        # update Lambdaz
        for (s in 1:R) {
            Lambdaz$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Az$mu[, s]^2 + 
                diag(Az$sigma[, , s])))
        }
        # update Az
        for (s in 1:R) {
            Az$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdaz$alpha[, s] * 
                Lambdaz$beta[, s]), Dz, Dz) + KzKz/sigma_g^2))
            Az$mu[, s] <- Az$sigma[, , s] %*% (tcrossprod(Kz, Gz$mu[s, , drop = FALSE])/sigma_g^2)
        }
        # update Gz
        Gz$sigma <- chol2inv(chol(diag(1/sigma_g^2, R, R) + (tcrossprod(Gx$mu, Gx$mu) + 
            Nx * Gx$sigma)/sigma_y^2))
        Gz$mu <- Gz$sigma %*% (crossprod(Az$mu, Kz)/sigma_g^2 + Gx$mu %*% Y/sigma_y^2)
    }
    
    state <- list(Lambdax = Lambdax, Ax = Ax, Lambdaz = Lambdaz, Az = Az, parameters = parameters)
}

In [ ]:
# S'il manque des Y dans la matrice, le modèle est en mode « semisupervisé »
kbmf1k1k_semisupervised_regression_variational_train <- function(Kx, Kz, Y, parameters) {
    set.seed(parameters$seed)
    
    Dx <- dim(Kx)[1]
    Nx <- dim(Kx)[2]
    Dz <- dim(Kz)[1]
    Nz <- dim(Kz)[2]
    R <- parameters$R
    sigma_g <- parameters$sigma_g
    sigma_y <- parameters$sigma_y
    
    Lambdax <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dx, R), beta = matrix(parameters$beta_lambda, 
        Dx, R))
    Ax <- list(mu = matrix(rnorm(Dx * R), Dx, R), sigma = array(diag(1, Dx, Dx), 
        c(Dx, Dx, R)))
    Gx <- list(mu = matrix(rnorm(R * Nx), R, Nx), sigma = array(diag(1, R, R), c(R, 
        R, Nx)))
    
    Lambdaz <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dz, R), beta = matrix(parameters$beta_lambda, 
        Dz, R))
    Az <- list(mu = matrix(rnorm(Dz * R), Dz, R), sigma = array(diag(1, Dz, Dz), 
        c(Dz, Dz, R)))
    Gz <- list(mu = matrix(rnorm(R * Nz), R, Nz), sigma = array(diag(1, R, R), c(R, 
        R, Nz)))
    
    KxKx <- tcrossprod(Kx, Kx)
    KzKz <- tcrossprod(Kz, Kz)
    
    for (iter in 1:parameters$iteration) {
        
        # update Lambdax
        for (s in 1:R) {
            Lambdax$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Ax$mu[, s]^2 + 
                diag(Ax$sigma[, , s])))
        }
        # update Ax
        for (s in 1:R) {
            Ax$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdax$alpha[, s] * 
                Lambdax$beta[, s]), Dx, Dx) + KxKx/sigma_g^2))
            Ax$mu[, s] <- Ax$sigma[, , s] %*% (tcrossprod(Kx, Gx$mu[s, , drop = FALSE])/sigma_g^2)
        }
        # update Gx
        for (i in 1:Nx) {
            indices <- which(is.na(Y[i, ]) == FALSE)
            Gx$sigma[, , i] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + (tcrossprod(Gz$mu[, 
                indices, drop = FALSE], Gz$mu[, indices, drop = FALSE]) + apply(Gz$sigma[, 
                , indices, drop = FALSE], 1:2, sum))/sigma_y^2))
            Gx$mu[, i] <- Gx$sigma[, , i] %*% (crossprod(Ax$mu, Kx[, i])/sigma_g^2 + 
                tcrossprod(Gz$mu[, indices, drop = FALSE], Y[i, indices, drop = FALSE])/sigma_y^2)
        }
        # update Lambdaz
        for (s in 1:R) {
            Lambdaz$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Az$mu[, s]^2 + 
                diag(Az$sigma[, , s])))
        }
        # update Az
        for (s in 1:R) {
            Az$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdaz$alpha[, s] * 
                Lambdaz$beta[, s]), Dz, Dz) + KzKz/sigma_g^2))
            Az$mu[, s] <- Az$sigma[, , s] %*% (tcrossprod(Kz, Gz$mu[s, , drop = FALSE])/sigma_g^2)
        }
        # update Gz
        for (j in 1:Nz) {
            indices <- which(is.na(Y[, j]) == FALSE)
            Gz$sigma[, , j] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + (tcrossprod(Gx$mu[, 
                indices, drop = FALSE], Gx$mu[, indices, drop = FALSE]) + apply(Gx$sigma[, 
                , indices, drop = FALSE], 1:2, sum))/sigma_y^2))
            Gz$mu[, j] <- Gz$sigma[, , j] %*% (crossprod(Az$mu, Kz[, j])/sigma_g^2 + 
                Gx$mu[, indices, drop = FALSE] %*% Y[indices, j, drop = FALSE]/sigma_y^2)
        }
    }
    
    state <- list(Lambdax = Lambdax, Ax = Ax, Lambdaz = Lambdaz, Az = Az, parameters = parameters)
}

### Fonctions de test du modèle

In [ ]:
# Mode supervisé
kbmf1k1k_supervised_regression_variational_test <- function(Kx, Kz, state) {
    Gx <- list(mu = crossprod(state$Ax$mu, Kx))
    
    Gz <- list(mu = crossprod(state$Az$mu, Kz))
    
    Y <- list(mu = crossprod(Gx$mu, Gz$mu))
    
    prediction <- list(Gx = Gx, Gz = Gz, Y = Y)
}

# Mode seimsupervisé
kbmf1k1k_semisupervised_regression_variational_test <- function(Kx, Kz, state) {
    Gx <- list(mu = crossprod(state$Ax$mu, Kx))
    
    Gz <- list(mu = crossprod(state$Az$mu, Kz))
    
    Y <- list(mu = crossprod(Gx$mu, Gz$mu))
    
    prediction <- list(Gx = Gx, Gz = Gz, Y = Y)
}

## Noyaux d'apprentissage multiple sur U et V

In [ ]:
# S'il ne manque pas de Y dans la matrice, le modèle est en mode « supervisé »
kbmf1mkl1mkl_supervised_regression_variational_train <- function(Kx, Kz, Y, parameters) {
    set.seed(parameters$seed)
    
    # initialisation des paramètres
    Dx <- dim(Kx)[1]
    Nx <- dim(Kx)[2]
    Px <- dim(Kx)[3]
    Dz <- dim(Kz)[1]
    Nz <- dim(Kz)[2]
    Pz <- dim(Kz)[3]
    R <- parameters$R
    sigma_g <- parameters$sigma_g
    sigma_h <- parameters$sigma_h
    sigma_y <- parameters$sigma_y
    
    # Lambda X
    Lambdax <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dx, R), beta = matrix(parameters$beta_lambda, 
        Dx, R))
    
    Ax <- list(mu = matrix(rnorm(Dx * R), Dx, R), sigma = array(diag(1, Dx, Dx), 
        c(Dx, Dx, R)))
    
    Gx <- list(mu = array(rnorm(R * Nx * Px), c(R, Nx, Px)), sigma = array(diag(1, 
        R, R), c(R, R, Px)))
    
    ## Apprentissage par noyau multiple
    etax <- list(alpha = matrix(parameters$alpha_eta + 0.5, Px, 1), beta = matrix(parameters$beta_eta, 
        Px, 1))
    
    ex <- list(mu = matrix(1, Px, 1), sigma = diag(1, Px, Px))
    Hx <- list(mu = matrix(rnorm(R * Nx), R, Nx), sigma = diag(1, R, R))
    
    # Lambda Z
    Lambdaz <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dz, R), beta = matrix(parameters$beta_lambda, 
        Dz, R))
    Az <- list(mu = matrix(rnorm(Dz * R), Dz, R), sigma = array(diag(1, Dz, Dz), 
        c(Dz, Dz, R)))
    Gz <- list(mu = array(rnorm(R * Nz * Pz), c(R, Nz, Pz)), sigma = array(diag(1, 
        R, R), c(R, R, Pz)))
    
    ## Apprentissage par noyau multiple
    etaz <- list(alpha = matrix(parameters$alpha_eta + 0.5, Pz, 1), beta = matrix(parameters$beta_eta, 
        Pz, 1))
    ez <- list(mu = matrix(1, Pz, 1), sigma = diag(1, Pz, Pz))
    Hz <- list(mu = matrix(rnorm(R * Nz), R, Nz), sigma = diag(1, R, R))
    
    # Calcul sur les noyaux préentrainement
    ## Pour réduire en complexité computationnel, les auteurs suggèrent de mettre en cache
    ## les noyaux
    # Cache sur Kx
    KxKx <- matrix(0, Dx, Dx)
    for (m in 1:Px) {
        KxKx <- KxKx + tcrossprod(Kx[, , m], Kx[, , m])
    }
    Kx <- matrix(Kx, Dx, Nx * Px)
    
    # Cache sur Kz
    KzKz <- matrix(0, Dz, Dz)
    for (n in 1:Pz) {
        KzKz <- KzKz + tcrossprod(Kz[, , n], Kz[, , n])
    }
    Kz <- matrix(Kz, Dz, Nz * Pz)
    
    
    # Entrainement du modèle (variationel)
    for (iter in 1:parameters$iteration) {
        # update Lambdax
        for (s in 1:R) {
            Lambdax$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Ax$mu[, s]^2 + 
                diag(Ax$sigma[, , s])))
        }
        # update Ax
        for (s in 1:R) {
            Ax$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdax$alpha[, s] * 
                Lambdax$beta[, s]), Dx, Dx) + KxKx/sigma_g^2))
            Ax$mu[, s] <- Ax$sigma[, , s] %*% (Kx %*% matrix(Gx$mu[s, , ], Nx * Px, 
                1)/sigma_g^2)
        }
        # update Gx
        for (m in 1:Px) {
            Gx$sigma[, , m] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + diag((ex$mu[m] * 
                ex$mu[m] + ex$sigma[m, m])/sigma_h^2, R, R)))
            Gx$mu[, , m] <- crossprod(Ax$mu, Kx[, ((m - 1) * Nx + 1):(m * Nx)])/sigma_g^2 + 
                ex$mu[m] * Hx$mu/sigma_h^2
            for (o in setdiff(1:Px, m)) {
                Gx$mu[, , m] <- Gx$mu[, , m] - (ex$mu[m] * ex$mu[o] + ex$sigma[m, 
                  o]) * Gx$mu[, , o]/sigma_h^2
            }
            Gx$mu[, , m] <- Gx$sigma[, , m] %*% Gx$mu[, , m]
        }
        # update etax
        etax$beta <- 1/(1/parameters$beta_eta + 0.5 * (ex$mu^2 + diag(ex$sigma)))
        
        # update ex
        ex$sigma <- diag(as.vector(etax$alpha * etax$beta))
        for (m in 1:Px) {
            for (o in 1:Px) {
                ex$sigma[m, o] <- ex$sigma[m, o] + (sum(Gx$mu[, , m] * Gx$mu[, , 
                  o]) + (m == o) * Nx * sum(diag(Gx$sigma[, , m])))/sigma_h^2
            }
        }
        
        ex$sigma <- chol2inv(chol(ex$sigma))
        for (m in 1:Px) {
            ex$mu[m] <- sum(Gx$mu[, , m] * Hx$mu)/sigma_h^2
        }
        ex$mu <- ex$sigma %*% ex$mu
        
        # update Hx
        Hx$sigma <- chol2inv(chol(diag(1/sigma_h^2, R, R) + (tcrossprod(Hz$mu, Hz$mu) + 
            Nz * Hz$sigma)/sigma_y^2))
        
        Hz$mu
        Hx$mu <- tcrossprod(Hz$mu, Y)/sigma_y^2
        for (m in 1:Px) {
            Hx$mu <- Hx$mu + ex$mu[m] * Gx$mu[, , m]/sigma_h^2
        }
        
        Hx$mu <- Hx$sigma %*% Hx$mu
        
        # update Lambdaz
        for (s in 1:R) {
            Lambdaz$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Az$mu[, s]^2 + 
                diag(Az$sigma[, , s])))
        }
        # update Az
        for (s in 1:R) {
            Az$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdaz$alpha[, s] * 
                Lambdaz$beta[, s]), Dz, Dz) + KzKz/sigma_g^2))
            Az$mu[, s] <- Az$sigma[, , s] %*% (Kz %*% matrix(Gz$mu[s, , ], Nz * Pz, 
                1)/sigma_g^2)
        }
        # update Gz
        for (n in 1:Pz) {
            Gz$sigma[, , n] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + diag((ez$mu[n] * 
                ez$mu[n] + ez$sigma[n, n])/sigma_h^2, R, R)))
            Gz$mu[, , n] <- crossprod(Az$mu, Kz[, ((n - 1) * Nz + 1):(n * Nz)])/sigma_g^2 + 
                ez$mu[n] * Hz$mu/sigma_h^2
            for (p in setdiff(1:Pz, n)) {
                Gz$mu[, , n] <- Gz$mu[, , n] - (ez$mu[n] * ez$mu[p] + ez$sigma[n, 
                  p]) * Gz$mu[, , p]/sigma_h^2
            }
            Gz$mu[, , n] <- Gz$sigma[, , n] %*% Gz$mu[, , n]
        }
        # update etaz
        etaz$beta <- 1/(1/parameters$beta_eta + 0.5 * (ez$mu^2 + diag(ez$sigma)))
        # update ez
        ez$sigma <- diag(as.vector(etaz$alpha * etaz$beta))
        for (n in 1:Pz) {
            for (p in 1:Pz) {
                ez$sigma[n, p] <- ez$sigma[n, p] + (sum(Gz$mu[, , n] * Gz$mu[, , 
                  p]) + (n == p) * Nz * sum(diag(Gz$sigma[, , n])))/sigma_h^2
            }
        }
        ez$sigma <- chol2inv(chol(ez$sigma))
        for (n in 1:Pz) {
            ez$mu[n] <- sum(Gz$mu[, , n] * Hz$mu)/sigma_h^2
        }
        ez$mu <- ez$sigma %*% ez$mu
        # update Hz
        Hz$sigma <- chol2inv(chol(diag(1/sigma_h^2, R, R) + (tcrossprod(Hx$mu, Hx$mu) + 
            Nx * Hx$sigma)/sigma_y^2))
        Hz$mu <- Hx$mu %*% Y/sigma_y^2
        for (n in 1:Pz) {
            Hz$mu <- Hz$mu + ez$mu[n] * Gz$mu[, , n]/sigma_h^2
        }
        Hz$mu <- Hz$sigma %*% Hz$mu
    }
    
    state <- list(Lambdax = Lambdax, Ax = Ax, etax = etax, ex = ex, Lambdaz = Lambdaz, 
        Az = Az, etaz = etaz, ez = ez, parameters = parameters)
}


# S'il manque des Y dans la matrice, le modèle est en mode « semisupervisé »
kbmf1mkl1mkl_semisupervised_regression_variational_train <- function(Kx, Kz, Y, parameters) {
    
    set.seed(parameters$seed)
    # Initialisation des paramètres
    Dx <- dim(Kx)[1]
    Nx <- dim(Kx)[2]
    Px <- dim(Kx)[3]
    Dz <- dim(Kz)[1]
    Nz <- dim(Kz)[2]
    Pz <- dim(Kz)[3]
    R <- parameters$R
    sigma_g <- parameters$sigma_g
    sigma_h <- parameters$sigma_h
    sigma_y <- parameters$sigma_y
    
    
    # Lambda X
    Lambdax <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dx, R), beta = matrix(parameters$beta_lambda, 
        Dx, R))
    Ax <- list(mu = matrix(rnorm(Dx * R), Dx, R), sigma = array(diag(1, Dx, Dx), 
        c(Dx, Dx, R)))
    Gx <- list(mu = array(rnorm(R * Nx * Px), c(R, Nx, Px)), sigma = array(diag(1, 
        R, R), c(R, R, Px)))
    etax <- list(alpha = matrix(parameters$alpha_eta + 0.5, Px, 1), beta = matrix(parameters$beta_eta, 
        Px, 1))
    ex <- list(mu = matrix(1, Px, 1), sigma = diag(1, Px, Px))
    Hx <- list(mu = matrix(rnorm(R * Nx), R, Nx), sigma = array(diag(1, R, R), c(R, 
        R, Nx)))
    
    # Lambda Z
    Lambdaz <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dz, R), beta = matrix(parameters$beta_lambda, 
        Dz, R))
    Az <- list(mu = matrix(rnorm(Dz * R), Dz, R), sigma = array(diag(1, Dz, Dz), 
        c(Dz, Dz, R)))
    Gz <- list(mu = array(rnorm(R * Nz * Pz), c(R, Nz, Pz)), sigma = array(diag(1, 
        R, R), c(R, R, Pz)))
    etaz <- list(alpha = matrix(parameters$alpha_eta + 0.5, Pz, 1), beta = matrix(parameters$beta_eta, 
        Pz, 1))
    ez <- list(mu = matrix(1, Pz, 1), sigma = diag(1, Pz, Pz))
    Hz <- list(mu = matrix(rnorm(R * Nz), R, Nz), sigma = array(diag(1, R, R), c(R, 
        R, Nz)))
    
    # Kernel pour X
    KxKx <- matrix(0, Dx, Dx)
    for (m in 1:Px) {
        KxKx <- KxKx + tcrossprod(Kx[, , m], Kx[, , m])
    }
    Kx <- matrix(Kx, Dx, Nx * Px)
    
    # Kernel pour Z
    KzKz <- matrix(0, Dz, Dz)
    for (n in 1:Pz) {
        KzKz <- KzKz + tcrossprod(Kz[, , n], Kz[, , n])
    }
    Kz <- matrix(Kz, Dz, Nz * Pz)
    
    # Mise à jour des paramètres
    for (iter in 1:parameters$iteration) {
        # update Lambdax
        for (s in 1:R) {
            Lambdax$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Ax$mu[, s]^2 + 
                diag(Ax$sigma[, , s])))
        }
        # update Ax
        for (s in 1:R) {
            Ax$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdax$alpha[, s] * 
                Lambdax$beta[, s]), Dx, Dx) + KxKx/sigma_g^2))
            Ax$mu[, s] <- Ax$sigma[, , s] %*% (Kx %*% matrix(Gx$mu[s, , ], Nx * Px, 
                1)/sigma_g^2)
        }
        # update Gx
        for (m in 1:Px) {
            Gx$sigma[, , m] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + diag((ex$mu[m] * 
                ex$mu[m] + ex$sigma[m, m])/sigma_h^2, R, R)))
            Gx$mu[, , m] <- crossprod(Ax$mu, Kx[, ((m - 1) * Nx + 1):(m * Nx)])/sigma_g^2 + 
                ex$mu[m] * Hx$mu/sigma_h^2
            for (o in setdiff(1:Px, m)) {
                Gx$mu[, , m] <- Gx$mu[, , m] - (ex$mu[m] * ex$mu[o] + ex$sigma[m, 
                  o]) * Gx$mu[, , o]/sigma_h^2
            }
            Gx$mu[, , m] <- Gx$sigma[, , m] %*% Gx$mu[, , m]
        }
        # update etax
        etax$beta <- 1/(1/parameters$beta_eta + 0.5 * (ex$mu^2 + diag(ex$sigma)))
        # update ex
        ex$sigma <- diag(as.vector(etax$alpha * etax$beta))
        for (m in 1:Px) {
            for (o in 1:Px) {
                ex$sigma[m, o] <- ex$sigma[m, o] + (sum(Gx$mu[, , m] * Gx$mu[, , 
                  o]) + (m == o) * Nx * sum(diag(Gx$sigma[, , m])))/sigma_h^2
            }
        }
        ex$sigma <- chol2inv(chol(ex$sigma))
        for (m in 1:Px) {
            ex$mu[m] <- sum(Gx$mu[, , m] * Hx$mu)/sigma_h^2
        }
        ex$mu <- ex$sigma %*% ex$mu
        # update Hx
        for (i in 1:Nx) {
            indices <- which(is.na(Y[i, ]) == FALSE)
            Hx$sigma[, , i] <- chol2inv(chol(diag(1/sigma_h^2, R, R) + (tcrossprod(Hz$mu[, 
                indices, drop = FALSE], Hz$mu[, indices, drop = FALSE]) + apply(Hz$sigma[, 
                , indices, drop = FALSE], 1:2, sum))/sigma_y^2))
            Hx$mu[, i] <- tcrossprod(Hz$mu[, indices, drop = FALSE], Y[i, indices, 
                drop = FALSE])/sigma_y^2
            for (m in 1:Px) {
                Hx$mu[, i] <- Hx$mu[, i] + ex$mu[m] * Gx$mu[, i, m]/sigma_h^2
            }
            Hx$mu[, i] <- Hx$sigma[, , i] %*% Hx$mu[, i]
        }
        
        # update Lambdaz
        for (s in 1:R) {
            Lambdaz$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Az$mu[, s]^2 + 
                diag(Az$sigma[, , s])))
        }
        # update Az
        
        for (s in 1:R) {
            Az$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdaz$alpha[, s] * 
                Lambdaz$beta[, s]), Dz, Dz) + KzKz/sigma_g^2))
            Az$mu[, s] <- Az$sigma[, , s] %*% (Kz %*% matrix(Gz$mu[s, , ], Nz * Pz, 
                1)/sigma_g^2)
        }
        # update Gz
        for (n in 1:Pz) {
            Gz$sigma[, , n] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + diag((ez$mu[n] * 
                ez$mu[n] + ez$sigma[n, n])/sigma_h^2, R, R)))
            Gz$mu[, , n] <- crossprod(Az$mu, Kz[, ((n - 1) * Nz + 1):(n * Nz)])/sigma_g^2 + 
                ez$mu[n] * Hz$mu/sigma_h^2
            for (p in setdiff(1:Pz, n)) {
                Gz$mu[, , n] <- Gz$mu[, , n] - (ez$mu[n] * ez$mu[p] + ez$sigma[n, 
                  p]) * Gz$mu[, , p]/sigma_h^2
            }
            Gz$mu[, , n] <- Gz$sigma[, , n] %*% Gz$mu[, , n]
        }
        # update etaz
        etaz$beta <- 1/(1/parameters$beta_eta + 0.5 * (ez$mu^2 + diag(ez$sigma)))
        # update ez
        ez$sigma <- diag(as.vector(etaz$alpha * etaz$beta))
        for (n in 1:Pz) {
            for (p in 1:Pz) {
                ez$sigma[n, p] <- ez$sigma[n, p] + (sum(Gz$mu[, , n] * Gz$mu[, , 
                  p]) + (n == p) * Nz * sum(diag(Gz$sigma[, , n])))/sigma_h^2
            }
        }
        ez$sigma <- chol2inv(chol(ez$sigma))
        for (n in 1:Pz) {
            ez$mu[n] <- sum(Gz$mu[, , n] * Hz$mu)/sigma_h^2
        }
        ez$mu <- ez$sigma %*% ez$mu
        # update Hz
        for (j in 1:Nz) {
            indices <- which(is.na(Y[, j]) == FALSE)
            Hz$sigma[, , j] <- chol2inv(chol(diag(1/sigma_h^2, R, R) + (tcrossprod(Hx$mu[, 
                indices, drop = FALSE], Hx$mu[, indices, drop = FALSE]) + apply(Hx$sigma[, 
                , indices, drop = FALSE], 1:2, sum))/sigma_y^2))
            Hz$mu[, j] <- Hx$mu[, indices, drop = FALSE] %*% Y[indices, j, drop = FALSE]/sigma_y^2
            for (n in 1:Pz) {
                Hz$mu[, j] <- Hz$mu[, j] + ez$mu[n] * Gz$mu[, j, n]/sigma_h^2
            }
            Hz$mu[, j] <- Hz$sigma[, , j] %*% Hz$mu[, j]
        }
    }
    # ?
    state <- list(Lambdax = Lambdax, Ax = Ax, etax = etax, ex = ex, Lambdaz = Lambdaz, 
        Az = Az, etaz = etaz, ez = ez, parameters = parameters)
}

**Fonctions de test du modèle**

In [ ]:
# Mode supervisé
kbmf1mkl1mkl_supervised_regression_variational_test <- function(Kx, Kz, state) {
    
    # initialisation des paramètres
    Nx <- dim(Kx)[2]
    Px <- dim(Kx)[3]
    Nz <- dim(Kz)[2]
    Pz <- dim(Kz)[3]
    R <- dim(state$Ax$mu)[2]
    
    Gx <- list(mu = array(0, c(R, Nx, Px)))
    for (m in 1:Px) {
        Gx$mu[, , m] <- crossprod(state$Ax$mu, Kx[, , m])
    }
    
    Hx <- list(mu = matrix(0, R, Nx))
    for (m in 1:Px) {
        Hx$mu <- Hx$mu + state$ex$mu[m] * Gx$mu[, , m]
    }
    
    Gz <- list(mu = array(0, c(R, Nz, Pz)))
    for (n in 1:Pz) {
        Gz$mu[, , n] <- crossprod(state$Az$mu, Kz[, , n])
    }
    
    Hz <- list(mu = matrix(0, R, Nz))
    for (n in 1:Pz) {
        Hz$mu <- Hz$mu + state$ez$mu[n] * Gz$mu[, , n]
    }
    
    Y <- list(mu = crossprod(Hx$mu, Hz$mu))
    
    prediction <- list(Gx = Gx, Hx = Hx, Gz = Gz, Hz = Hz, Y = Y)
}

In [ ]:
# Mode semisupervisé
kbmf1mkl1mkl_semisupervised_regression_variational_test <- function(Kx, Kz, state) {
    Nx <- dim(Kx)[2]
    Px <- dim(Kx)[3]
    Nz <- dim(Kz)[2]
    Pz <- dim(Kz)[3]
    R <- dim(state$Ax$mu)[2]
    
    Gx <- list(mu = array(0, c(R, Nx, Px)))
    for (m in 1:Px) {
        Gx$mu[, , m] <- crossprod(state$Ax$mu, Kx[, , m])
    }
    Hx <- list(mu = matrix(0, R, Nx))
    for (m in 1:Px) {
        Hx$mu <- Hx$mu + state$ex$mu[m] * Gx$mu[, , m]
    }
    
    Gz <- list(mu = array(0, c(R, Nz, Pz)))
    for (n in 1:Pz) {
        Gz$mu[, , n] <- crossprod(state$Az$mu, Kz[, , n])
    }
    Hz <- list(mu = matrix(0, R, Nz))
    for (n in 1:Pz) {
        Hz$mu <- Hz$mu + state$ez$mu[n] * Gz$mu[, , n]
    }
    
    Y <- list(mu = crossprod(Hx$mu, Hz$mu))
    
    prediction <- list(Gx = Gx, Hx = Hx, Gz = Gz, Hz = Hz, Y = Y)
}

## Multiples noyaux sur U, un noyau sur V

In [ ]:
# S'il manque des Y dans la matrice, le modèle est en mode « semisupervisé »
kbmf1mkl1k_supervised_regression_variational_train <- function(Kx, Kz, Y, parameters) {
    set.seed(parameters$seed)
    # Initialisation des paramètres
    Dx <- dim(Kx)[1]
    Nx <- dim(Kx)[2]
    Px <- dim(Kx)[3]
    Dz <- dim(Kz)[1]
    Nz <- dim(Kz)[2]
    R <- parameters$R
    sigma_g <- parameters$sigma_g
    sigma_h <- parameters$sigma_h
    sigma_y <- parameters$sigma_y
    
    # Lambda X
    Lambdax <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dx, R), beta = matrix(parameters$beta_lambda, 
        Dx, R))
    Ax <- list(mu = matrix(rnorm(Dx * R), Dx, R), sigma = array(diag(1, Dx, Dx), 
        c(Dx, Dx, R)))
    Gx <- list(mu = array(rnorm(R * Nx * Px), c(R, Nx, Px)), sigma = array(diag(1, 
        R, R), c(R, R, Px)))
    #######
    etax <- list(alpha = matrix(parameters$alpha_eta + 0.5, Px, 1), beta = matrix(parameters$beta_eta, 
        Px, 1))
    ex <- list(mu = matrix(1, Px, 1), sigma = diag(1, Px, Px))
    Hx <- list(mu = matrix(rnorm(R * Nx), R, Nx), sigma = diag(1, R, R))
    
    # Lambda Z
    Lambdaz <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dz, R), beta = matrix(parameters$beta_lambda, 
        Dz, R))
    Az <- list(mu = matrix(rnorm(Dz * R), Dz, R), sigma = array(diag(1, Dz, Dz), 
        c(Dz, Dz, R)))
    Gz <- list(mu = matrix(rnorm(R * Nz), R, Nz), sigma = diag(1, R, R))
    
    # Mise en cache des noyaux
    KxKx <- matrix(0, Dx, Dx)
    for (m in 1:Px) {
        KxKx <- KxKx + tcrossprod(Kx[, , m], Kx[, , m])
    }
    Kx <- matrix(Kx, Dx, Nx * Px)
    
    KzKz <- tcrossprod(Kz, Kz)
    
    # Mise à jour des paramètres
    for (iter in 1:parameters$iteration) {
        # update Lambdax
        for (s in 1:R) {
            Lambdax$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Ax$mu[, s]^2 + 
                diag(Ax$sigma[, , s])))
        }
        # update Ax
        for (s in 1:R) {
            Ax$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdax$alpha[, s] * 
                Lambdax$beta[, s]), Dx, Dx) + KxKx/sigma_g^2))
            Ax$mu[, s] <- Ax$sigma[, , s] %*% (Kx %*% matrix(Gx$mu[s, , ], Nx * Px, 
                1)/sigma_g^2)
        }
        # update Gx
        for (m in 1:Px) {
            Gx$sigma[, , m] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + diag((ex$mu[m] * 
                ex$mu[m] + ex$sigma[m, m])/sigma_h^2, R, R)))
            Gx$mu[, , m] <- crossprod(Ax$mu, Kx[, ((m - 1) * Nx + 1):(m * Nx)])/sigma_g^2 + 
                ex$mu[m] * Hx$mu/sigma_h^2
            for (o in setdiff(1:Px, m)) {
                Gx$mu[, , m] <- Gx$mu[, , m] - (ex$mu[m] * ex$mu[o] + ex$sigma[m, 
                  o]) * Gx$mu[, , o]/sigma_h^2
            }
            Gx$mu[, , m] <- Gx$sigma[, , m] %*% Gx$mu[, , m]
        }
        
        # update etax
        etax$beta <- 1/(1/parameters$beta_eta + 0.5 * (ex$mu^2 + diag(ex$sigma)))
        
        # update ex
        ex$sigma <- diag(as.vector(etax$alpha * etax$beta))
        for (m in 1:Px) {
            for (o in 1:Px) {
                ex$sigma[m, o] <- ex$sigma[m, o] + (sum(Gx$mu[, , m] * Gx$mu[, , 
                  o]) + (m == o) * Nx * sum(diag(Gx$sigma[, , m])))/sigma_h^2
            }
        }
        ex$sigma <- chol2inv(chol(ex$sigma))
        for (m in 1:Px) {
            ex$mu[m] <- sum(Gx$mu[, , m] * Hx$mu)/sigma_h^2
        }
        ex$mu <- ex$sigma %*% ex$mu
        
        # update Hx
        Hx$sigma <- chol2inv(chol(diag(1/sigma_h^2, R, R) + (tcrossprod(Gz$mu, Gz$mu) + 
            Nz * Gz$sigma)/sigma_y^2))
        Hx$mu <- tcrossprod(Gz$mu, Y)/sigma_y^2
        for (m in 1:Px) {
            Hx$mu <- Hx$mu + ex$mu[m] * Gx$mu[, , m]/sigma_h^2
        }
        Hx$mu <- Hx$sigma %*% Hx$mu
        
        # update Lambdaz
        for (s in 1:R) {
            Lambdaz$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Az$mu[, s]^2 + 
                diag(Az$sigma[, , s])))
        }
        
        # update Az
        for (s in 1:R) {
            Az$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdaz$alpha[, s] * 
                Lambdaz$beta[, s]), Dz, Dz) + KzKz/sigma_g^2))
            Az$mu[, s] <- Az$sigma[, , s] %*% (tcrossprod(Kz, Gz$mu[s, , drop = FALSE])/sigma_g^2)
        }
        
        # update Gz
        Gz$sigma <- chol2inv(chol(diag(1/sigma_g^2, R, R) + (tcrossprod(Hx$mu, Hx$mu) + 
            Nx * Hx$sigma)/sigma_y^2))
        Gz$mu <- Gz$sigma %*% (crossprod(Az$mu, Kz)/sigma_g^2 + Hx$mu %*% Y/sigma_y^2)
    }
    
    state <- list(Lambdax = Lambdax, Ax = Ax, etax = etax, ex = ex, Lambdaz = Lambdaz, 
        Az = Az, parameters = parameters)
}



In [ ]:
# S'il manque des Y dans la matrice, le modèle est en mode « semisupervisé »
kbmf1mkl1k_semisupervised_regression_variational_train <- function(Kx, Kz, Y, parameters) {
    set.seed(parameters$seed)
    # Initialisation des paramètres
    Dx <- dim(Kx)[1]
    Nx <- dim(Kx)[2]
    Px <- dim(Kx)[3]
    Dz <- dim(Kz)[1]
    Nz <- dim(Kz)[2]
    R <- parameters$R
    sigma_g <- parameters$sigma_g
    sigma_h <- parameters$sigma_h
    sigma_y <- parameters$sigma_y
    
    # Lambda X
    Lambdax <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dx, R), beta = matrix(parameters$beta_lambda, 
        Dx, R))
    Ax <- list(mu = matrix(rnorm(Dx * R), Dx, R), sigma = array(diag(1, Dx, Dx), 
        c(Dx, Dx, R)))
    Gx <- list(mu = array(rnorm(R * Nx * Px), c(R, Nx, Px)), sigma = array(diag(1, 
        R, R), c(R, R, Px)))
    etax <- list(alpha = matrix(parameters$alpha_eta + 0.5, Px, 1), beta = matrix(parameters$beta_eta, 
        Px, 1))
    ex <- list(mu = matrix(1, Px, 1), sigma = diag(1, Px, Px))
    Hx <- list(mu = matrix(rnorm(R * Nx), R, Nx), sigma = array(diag(1, R, R), c(R, 
        R, Nx)))
    
    # Lambda Z
    Lambdaz <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dz, R), beta = matrix(parameters$beta_lambda, 
        Dz, R))
    Az <- list(mu = matrix(rnorm(Dz * R), Dz, R), sigma = array(diag(1, Dz, Dz), 
        c(Dz, Dz, R)))
    Gz <- list(mu = matrix(rnorm(R * Nz), R, Nz), sigma = array(diag(1, R, R), c(R, 
        R, Nz)))
    
    # Mise en cache des noyaux
    KxKx <- matrix(0, Dx, Dx)
    for (m in 1:Px) {
        KxKx <- KxKx + tcrossprod(Kx[, , m], Kx[, , m])
    }
    Kx <- matrix(Kx, Dx, Nx * Px)
    
    KzKz <- tcrossprod(Kz, Kz)
    
    # Mise à jour des paramètres
    for (iter in 1:parameters$iteration) {
        # update Lambdax
        for (s in 1:R) {
            Lambdax$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Ax$mu[, s]^2 + 
                diag(Ax$sigma[, , s])))
        }
        # update Ax
        for (s in 1:R) {
            Ax$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdax$alpha[, s] * 
                Lambdax$beta[, s]), Dx, Dx) + KxKx/sigma_g^2))
            Ax$mu[, s] <- Ax$sigma[, , s] %*% (Kx %*% matrix(Gx$mu[s, , ], Nx * Px, 
                1)/sigma_g^2)
        }
        # update Gx
        for (m in 1:Px) {
            Gx$sigma[, , m] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + diag((ex$mu[m] * 
                ex$mu[m] + ex$sigma[m, m])/sigma_h^2, R, R)))
            Gx$mu[, , m] <- crossprod(Ax$mu, Kx[, ((m - 1) * Nx + 1):(m * Nx)])/sigma_g^2 + 
                ex$mu[m] * Hx$mu/sigma_h^2
            for (o in setdiff(1:Px, m)) {
                Gx$mu[, , m] <- Gx$mu[, , m] - (ex$mu[m] * ex$mu[o] + ex$sigma[m, 
                  o]) * Gx$mu[, , o]/sigma_h^2
            }
            Gx$mu[, , m] <- Gx$sigma[, , m] %*% Gx$mu[, , m]
        }
        # update etax
        etax$beta <- 1/(1/parameters$beta_eta + 0.5 * (ex$mu^2 + diag(ex$sigma)))
        # update ex
        ex$sigma <- diag(as.vector(etax$alpha * etax$beta))
        for (m in 1:Px) {
            for (o in 1:Px) {
                ex$sigma[m, o] <- ex$sigma[m, o] + (sum(Gx$mu[, , m] * Gx$mu[, , 
                  o]) + (m == o) * Nx * sum(diag(Gx$sigma[, , m])))/sigma_h^2
            }
        }
        ex$sigma <- chol2inv(chol(ex$sigma))
        for (m in 1:Px) {
            ex$mu[m] <- sum(Gx$mu[, , m] * Hx$mu)/sigma_h^2
        }
        ex$mu <- ex$sigma %*% ex$mu
        # update Hx
        for (i in 1:Nx) {
            indices <- which(is.na(Y[i, ]) == FALSE)
            Hx$sigma[, , i] <- chol2inv(chol(diag(1/sigma_h^2, R, R) + (tcrossprod(Gz$mu[, 
                indices, drop = FALSE], Gz$mu[, indices, drop = FALSE]) + apply(Gz$sigma[, 
                , indices, drop = FALSE], 1:2, sum))/sigma_y^2))
            Hx$mu[, i] <- tcrossprod(Gz$mu[, indices, drop = FALSE], Y[i, indices, 
                drop = FALSE])/sigma_y^2
            for (m in 1:Px) {
                Hx$mu[, i] <- Hx$mu[, i] + ex$mu[m] * Gx$mu[, i, m]/sigma_h^2
            }
            Hx$mu[, i] <- Hx$sigma[, , i] %*% Hx$mu[, i]
        }
        
        # update Lambdaz
        for (s in 1:R) {
            Lambdaz$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Az$mu[, s]^2 + 
                diag(Az$sigma[, , s])))
        }
        # update Az
        for (s in 1:R) {
            Az$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdaz$alpha[, s] * 
                Lambdaz$beta[, s]), Dz, Dz) + KzKz/sigma_g^2))
            Az$mu[, s] <- Az$sigma[, , s] %*% (tcrossprod(Kz, Gz$mu[s, , drop = FALSE])/sigma_g^2)
        }
        # update Gz
        for (j in 1:Nz) {
            indices <- which(is.na(Y[, j]) == FALSE)
            Gz$sigma[, , j] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + (tcrossprod(Hx$mu[, 
                indices, drop = FALSE], Hx$mu[, indices, drop = FALSE]) + apply(Hx$sigma[, 
                , indices, drop = FALSE], 1:2, sum))/sigma_y^2))
            Gz$mu[, j] <- Gz$sigma[, , j] %*% (crossprod(Az$mu, Kz[, j])/sigma_g^2 + 
                Hx$mu[, indices, drop = FALSE] %*% Y[indices, j, drop = FALSE]/sigma_y^2)
        }
    }
    
    state <- list(Lambdax = Lambdax, Ax = Ax, etax = etax, ex = ex, Lambdaz = Lambdaz, 
        Az = Az, parameters = parameters)
}

**Fonction de test**

In [ ]:
# Mode supervisé
kbmf1mkl1k_supervised_regression_variational_test <- function(Kx, Kz, state) {
    # Initialisation des paramètres
    Nx <- dim(Kx)[2]
    Px <- dim(Kx)[3]
    Nz <- dim(Kz)[2]
    R <- dim(state$Ax$mu)[2]
    
    Gx <- list(mu = array(0, c(R, Nx, Px)))
    for (m in 1:Px) {
        Gx$mu[, , m] <- crossprod(state$Ax$mu, Kx[, , m])
    }
    Hx <- list(mu = matrix(0, R, Nx))
    for (m in 1:Px) {
        Hx$mu <- Hx$mu + state$ex$mu[m] * Gx$mu[, , m]
    }
    
    Gz <- list(mu = crossprod(state$Az$mu, Kz))
    
    Y <- list(mu = crossprod(Hx$mu, Gz$mu))
    
    prediction <- list(Gx = Gx, Hx = Hx, Gz = Gz, Y = Y)
}



In [ ]:
# Mode semisupervisé
kbmf1mkl1k_semisupervised_regression_variational_test <- function(Kx, Kz, state) {
    # Initialisation des paramètres
    Nx <- dim(Kx)[2]
    Px <- dim(Kx)[3]
    Nz <- dim(Kz)[2]
    R <- dim(state$Ax$mu)[2]
    
    Gx <- list(mu = array(0, c(R, Nx, Px)))
    for (m in 1:Px) {
        Gx$mu[, , m] <- crossprod(state$Ax$mu, Kx[, , m])
    }
    Hx <- list(mu = matrix(0, R, Nx))
    for (m in 1:Px) {
        Hx$mu <- Hx$mu + state$ex$mu[m] * Gx$mu[, , m]
    }
    
    Gz <- list(mu = crossprod(state$Az$mu, Kz))
    
    Y <- list(mu = crossprod(Hx$mu, Gz$mu))
    
    prediction <- list(Gx = Gx, Hx = Hx, Gz = Gz, Y = Y)
}

## Un noyau sur U, multiples noyaux sur V

In [ ]:
# Supervisé
kbmf1k1mkl_supervised_regression_variational_train <- function(Kx, Kz, Y, parameters) {
    set.seed(parameters$seed)
    # Initialisation des paramètres
    Dx <- dim(Kx)[1]
    Nx <- dim(Kx)[2]
    Dz <- dim(Kz)[1]
    Nz <- dim(Kz)[2]
    Pz <- dim(Kz)[3]
    R <- parameters$R
    sigma_g <- parameters$sigma_g
    sigma_h <- parameters$sigma_h
    sigma_y <- parameters$sigma_y
    
    # Lambda X
    Lambdax <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dx, R), beta = matrix(parameters$beta_lambda, 
        Dx, R))
    Ax <- list(mu = matrix(rnorm(Dx * R), Dx, R), sigma = array(diag(1, Dx, Dx), 
        c(Dx, Dx, R)))
    Gx <- list(mu = matrix(rnorm(R * Nx), R, Nx), sigma = diag(1, R, R))
    
    # Lambda Z
    Lambdaz <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dz, R), beta = matrix(parameters$beta_lambda, 
        Dz, R))
    Az <- list(mu = matrix(rnorm(Dz * R), Dz, R), sigma = array(diag(1, Dz, Dz), 
        c(Dz, Dz, R)))
    Gz <- list(mu = array(rnorm(R * Nz * Pz), c(R, Nz, Pz)), sigma = array(diag(1, 
        R, R), c(R, R, Pz)))
    etaz <- list(alpha = matrix(parameters$alpha_eta + 0.5, Pz, 1), beta = matrix(parameters$beta_eta, 
        Pz, 1))
    ez <- list(mu = matrix(1, Pz, 1), sigma = diag(1, Pz, Pz))
    Hz <- list(mu = matrix(rnorm(R * Nz), R, Nz), sigma = diag(1, R, R))
    
    # Kernels
        # Kernels X
    KxKx <- tcrossprod(Kx, Kx)
        # Kernels Z
    KzKz <- matrix(0, Dz, Dz)
    for (n in 1:Pz) {
        KzKz <- KzKz + tcrossprod(Kz[, , n], Kz[, , n])
    }
    Kz <- matrix(Kz, Dz, Nz * Pz)
    
    # Mise à jour des paramètres
    for (iter in 1:parameters$iteration) {
        # update Lambdax
        for (s in 1:R) {
            Lambdax$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Ax$mu[, s]^2 + 
                diag(Ax$sigma[, , s])))
        }
        # update Ax
        for (s in 1:R) {
            Ax$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdax$alpha[, s] * 
                Lambdax$beta[, s]), Dx, Dx) + KxKx/sigma_g^2))
            Ax$mu[, s] <- Ax$sigma[, , s] %*% (tcrossprod(Kx, Gx$mu[s, , drop = FALSE])/sigma_g^2)
        }
        # update Gx
        Gx$sigma <- chol2inv(chol(diag(1/sigma_g^2, R, R) + (tcrossprod(Hz$mu, Hz$mu) + 
            Nz * Hz$sigma)/sigma_y^2))
        Gx$mu <- Gx$sigma %*% (crossprod(Ax$mu, Kx)/sigma_g^2 + tcrossprod(Hz$mu, 
            Y)/sigma_y^2)
        
        # update Lambdaz
        for (s in 1:R) {
            Lambdaz$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Az$mu[, s]^2 + 
                diag(Az$sigma[, , s])))
        }
        # update Az
        for (s in 1:R) {
            Az$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdaz$alpha[, s] * 
                Lambdaz$beta[, s]), Dz, Dz) + KzKz/sigma_g^2))
            Az$mu[, s] <- Az$sigma[, , s] %*% (Kz %*% matrix(Gz$mu[s, , ], Nz * Pz, 
                1)/sigma_g^2)
        }
        # update Gz
        for (n in 1:Pz) {
            Gz$sigma[, , n] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + diag((ez$mu[n] * 
                ez$mu[n] + ez$sigma[n, n])/sigma_h^2, R, R)))
            Gz$mu[, , n] <- crossprod(Az$mu, Kz[, ((n - 1) * Nz + 1):(n * Nz)])/sigma_g^2 + 
                ez$mu[n] * Hz$mu/sigma_h^2
            for (p in setdiff(1:Pz, n)) {
                Gz$mu[, , n] <- Gz$mu[, , n] - (ez$mu[n] * ez$mu[p] + ez$sigma[n, 
                  p]) * Gz$mu[, , p]/sigma_h^2
            }
            Gz$mu[, , n] <- Gz$sigma[, , n] %*% Gz$mu[, , n]
        }
        # update etaz
        etaz$beta <- 1/(1/parameters$beta_eta + 0.5 * (ez$mu^2 + diag(ez$sigma)))
        # update ez
        ez$sigma <- diag(as.vector(etaz$alpha * etaz$beta))
        for (n in 1:Pz) {
            for (p in 1:Pz) {
                ez$sigma[n, p] <- ez$sigma[n, p] + (sum(Gz$mu[, , n] * Gz$mu[, , 
                  p]) + (n == p) * Nz * sum(diag(Gz$sigma[, , n])))/sigma_h^2
            }
        }
        ez$sigma <- chol2inv(chol(ez$sigma))
        for (n in 1:Pz) {
            ez$mu[n] <- sum(Gz$mu[, , n] * Hz$mu)/sigma_h^2
        }
        ez$mu <- ez$sigma %*% ez$mu
        # update Hz
        Hz$sigma <- chol2inv(chol(diag(1/sigma_h^2, R, R) + (tcrossprod(Gx$mu, Gx$mu) + 
            Nx * Gx$sigma)/sigma_y^2))
        Hz$mu <- Gx$mu %*% Y/sigma_y^2
        for (n in 1:Pz) {
            Hz$mu <- Hz$mu + ez$mu[n] * Gz$mu[, , n]/sigma_h^2
        }
        Hz$mu <- Hz$sigma %*% Hz$mu
    }
    
    state <- list(Lambdax = Lambdax, Ax = Ax, Lambdaz = Lambdaz, Az = Az, etaz = etaz, 
        ez = ez, parameters = parameters)
}



In [ ]:
kbmf1k1mkl_semisupervised_regression_variational_train <- function(Kx, Kz, Y, parameters) {
    set.seed(parameters$seed)
    # Initialisation des paramètres
    Dx <- dim(Kx)[1]
    Nx <- dim(Kx)[2]
    Dz <- dim(Kz)[1]
    Nz <- dim(Kz)[2]
    Pz <- dim(Kz)[3]
    R <- parameters$R
    sigma_g <- parameters$sigma_g
    sigma_h <- parameters$sigma_h
    sigma_y <- parameters$sigma_y
    
    # Lambda X
    Lambdax <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dx, R), beta = matrix(parameters$beta_lambda, 
        Dx, R))
    Ax <- list(mu = matrix(rnorm(Dx * R), Dx, R), sigma = array(diag(1, Dx, Dx), 
        c(Dx, Dx, R)))
    Gx <- list(mu = matrix(rnorm(R * Nx), R, Nx), sigma = array(diag(1, R, R), c(R, 
        R, Nx)))
    
    # Lambda Z
    Lambdaz <- list(alpha = matrix(parameters$alpha_lambda + 0.5, Dz, R), beta = matrix(parameters$beta_lambda, 
        Dz, R))
    Az <- list(mu = matrix(rnorm(Dz * R), Dz, R), sigma = array(diag(1, Dz, Dz), 
        c(Dz, Dz, R)))
    Gz <- list(mu = array(rnorm(R * Nz * Pz), c(R, Nz, Pz)), sigma = array(diag(1, 
        R, R), c(R, R, Pz)))
    etaz <- list(alpha = matrix(parameters$alpha_eta + 0.5, Pz, 1), beta = matrix(parameters$beta_eta, 
        Pz, 1))
    ez <- list(mu = matrix(1, Pz, 1), sigma = diag(1, Pz, Pz))
    Hz <- list(mu = matrix(rnorm(R * Nz), R, Nz), sigma = array(diag(1, R, R), c(R, 
        R, Nz)))
    
    # Mise en cache des kernels
    KxKx <- tcrossprod(Kx, Kx)
    
    KzKz <- matrix(0, Dz, Dz)
    for (n in 1:Pz) {
        KzKz <- KzKz + tcrossprod(Kz[, , n], Kz[, , n])
    }
    Kz <- matrix(Kz, Dz, Nz * Pz)
    
    # Mise à jour des paramètres
    for (iter in 1:parameters$iteration) {
        # update Lambdax
        for (s in 1:R) {
            Lambdax$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Ax$mu[, s]^2 + 
                diag(Ax$sigma[, , s])))
        }
        # update Ax
        for (s in 1:R) {
            Ax$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdax$alpha[, s] * 
                Lambdax$beta[, s]), Dx, Dx) + KxKx/sigma_g^2))
            Ax$mu[, s] <- Ax$sigma[, , s] %*% (tcrossprod(Kx, Gx$mu[s, , drop = FALSE])/sigma_g^2)
        }
        # update Gx
        for (i in 1:Nx) {
            indices <- which(is.na(Y[i, ]) == FALSE)
            Gx$sigma[, , i] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + (tcrossprod(Hz$mu[, 
                indices, drop = FALSE], Hz$mu[, indices, drop = FALSE]) + apply(Hz$sigma[, 
                , indices, drop = FALSE], 1:2, sum))/sigma_y^2))
            Gx$mu[, i] <- Gx$sigma[, , i] %*% (crossprod(Ax$mu, Kx[, i])/sigma_g^2 + 
                tcrossprod(Hz$mu[, indices, drop = FALSE], Y[i, indices, drop = FALSE])/sigma_y^2)
        }
        
        # update Lambdaz
        for (s in 1:R) {
            Lambdaz$beta[, s] <- 1/(1/parameters$beta_lambda + 0.5 * (Az$mu[, s]^2 + 
                diag(Az$sigma[, , s])))
        }
        # update Az
        for (s in 1:R) {
            Az$sigma[, , s] <- chol2inv(chol(diag(as.vector(Lambdaz$alpha[, s] * 
                Lambdaz$beta[, s]), Dz, Dz) + KzKz/sigma_g^2))
            Az$mu[, s] <- Az$sigma[, , s] %*% (Kz %*% matrix(Gz$mu[s, , ], Nz * Pz, 
                1)/sigma_g^2)
        }
        # update Gz
        for (n in 1:Pz) {
            Gz$sigma[, , n] <- chol2inv(chol(diag(1/sigma_g^2, R, R) + diag((ez$mu[n] * 
                ez$mu[n] + ez$sigma[n, n])/sigma_h^2, R, R)))
            Gz$mu[, , n] <- crossprod(Az$mu, Kz[, ((n - 1) * Nz + 1):(n * Nz)])/sigma_g^2 + 
                ez$mu[n] * Hz$mu/sigma_h^2
            for (p in setdiff(1:Pz, n)) {
                Gz$mu[, , n] <- Gz$mu[, , n] - (ez$mu[n] * ez$mu[p] + ez$sigma[n, 
                  p]) * Gz$mu[, , p]/sigma_h^2
            }
            Gz$mu[, , n] <- Gz$sigma[, , n] %*% Gz$mu[, , n]
        }
        # update etaz
        etaz$beta <- 1/(1/parameters$beta_eta + 0.5 * (ez$mu^2 + diag(ez$sigma)))
        # update ez
        ez$sigma <- diag(as.vector(etaz$alpha * etaz$beta))
        for (n in 1:Pz) {
            for (p in 1:Pz) {
                ez$sigma[n, p] <- ez$sigma[n, p] + (sum(Gz$mu[, , n] * Gz$mu[, , 
                  p]) + (n == p) * Nz * sum(diag(Gz$sigma[, , n])))/sigma_h^2
            }
        }
        ez$sigma <- chol2inv(chol(ez$sigma))
        for (n in 1:Pz) {
            ez$mu[n] <- sum(Gz$mu[, , n] * Hz$mu)/sigma_h^2
        }
        ez$mu <- ez$sigma %*% ez$mu
        # update Hz
        for (j in 1:Nz) {
            indices <- which(is.na(Y[, j]) == FALSE)
            Hz$sigma[, , j] <- chol2inv(chol(diag(1/sigma_h^2, R, R) + (tcrossprod(Gx$mu[, 
                indices, drop = FALSE], Gx$mu[, indices, drop = FALSE]) + apply(Gx$sigma[, 
                , indices, drop = FALSE], 1:2, sum))/sigma_y^2))
            Hz$mu[, j] <- Gx$mu[, indices, drop = FALSE] %*% Y[indices, j, drop = FALSE]/sigma_y^2
            for (n in 1:Pz) {
                Hz$mu[, j] <- Hz$mu[, j] + ez$mu[n] * Gz$mu[, j, n]/sigma_h^2
            }
            Hz$mu[, j] <- Hz$sigma[, , j] %*% Hz$mu[, j]
        }
    }
    
    state <- list(Lambdax = Lambdax, Ax = Ax, Lambdaz = Lambdaz, Az = Az, etaz = etaz, 
        ez = ez, parameters = parameters)
}

In [ ]:
# Mode supervisé
kbmf1k1mkl_supervised_regression_variational_test <- function(Kx, Kz, state) {
    # Initialisation des paramètres
    Nz <- dim(Kz)[2]
    Pz <- dim(Kz)[3]
    R <- dim(state$Ax$mu)[2]
    
    Gx <- list(mu = crossprod(state$Ax$mu, Kx))
    
    Gz <- list(mu = array(0, c(R, Nz, Pz)))
    for (n in 1:Pz) {
      Gz$mu[, , n] <- crossprod(state$Az$mu, Kz[, , n])
    }
    Hz <- list(mu = matrix(0, R, Nz))
    for (n in 1:Pz) {
      Hz$mu <- Hz$mu + state$ez$mu[n] * Gz$mu[, , n]
    }
    
    Y <- list(mu = crossprod(Gx$mu, Hz$mu))
    
    prediction <- list(Gx = Gx,
                       Gz = Gz,
                       Hz = Hz,
                       Y = Y)
  }



In [ ]:
# Mode semisupervisé
kbmf1k1mkl_semisupervised_regression_variational_test <- function(Kx, Kz, state) {
    # Initialisation des paramètres
    Nz <- dim(Kz)[2]
    Pz <- dim(Kz)[3]
    R <- dim(state$Ax$mu)[2]
    
    Gx <- list(mu = crossprod(state$Ax$mu, Kx))
    
    Gz <- list(mu = array(0, c(R, Nz, Pz)))
    for (n in 1:Pz) {
      Gz$mu[, , n] <- crossprod(state$Az$mu, Kz[, , n])
    }
    Hz <- list(mu = matrix(0, R, Nz))
    for (n in 1:Pz) {
      Hz$mu <- Hz$mu + state$ez$mu[n] * Gz$mu[, , n]
    }
    
    Y <- list(mu = crossprod(Gx$mu, Hz$mu))
    
    prediction <- list(Gx = Gx,
                       Gz = Gz,
                       Hz = Hz,
                       Y = Y)
  }

## Entrainement du modèle

*Le modèle de régression est ici utilisé*

In [ ]:
kbmf_regression_train <- function(Kx, Kz, Y, R, iter, varargin) {
    
    # ====================================================
    # Fonction d'entrainement du modèle
    # Paramètres en entrée  
    # Kx
    # Kz
    # Y
    # R
    # iter
    # varargin
    # 
    # Paramètres en sortie
    # 
    # 
    #
    # ====================================================
    
    
    # is_supervised <- all(!is.na(Y))
    is_supervised <- all(Y > 0)
    
    # Initialisation des paramètres
    Px <- dim(Kx)[3]
    Pz <- dim(Kz)[3]
    
    parameters <- list()
    
    parameters$alpha_lambda <- 1
    parameters$beta_lambda <- 1
    
    if (Px > 1 || Pz > 1) {
        parameters$alpha_eta <- 1
        parameters$beta_eta <- 1
    }
    parameters$iteration <- iter  # Automatiser dans la fonction
    parameters$progress <- 1  # Qu'est-ce que ça veut dire ? 
    parameters$R <- R  # c'est quoi ?
    parameters$seed <- 1606
    parameters$sigma_g <- 0.1  # ??
    
    if (Px > 1 || Pz > 1) {
        parameters$sigma_h <- 0.1
    }
    parameters$sigma_y <- 1
    
    # Choix du modèle (multi kernel learning pour X ou Z, kernel learning pour X ou Z)
    if (is_supervised == 1) {
        if (Px == 1 && Pz == 1) {
            train_function <- kbmf1k1k_supervised_regression_variational_train
            test_function <- kbmf1k1k_supervised_regression_variational_test
        }
        if (Px > 1 && Pz == 1) {
            train_function <- kbmf1mkl1k_supervised_regression_variational_train
            test_function <- kbmf1mkl1k_supervised_regression_variational_test
        }
        if (Px == 1 && Pz > 1) {
            train_function <- kbmf1k1mkl_supervised_regression_variational_train
            test_function <- kbmf1k1mkl_supervised_regression_variational_test
        }
        if (Px > 1 && Pz > 1) {
            train_function <- kbmf1mkl1mkl_supervised_regression_variational_train
            test_function <- kbmf1mkl1mkl_supervised_regression_variational_test
        }
    } else {
        if (Px == 1 && Pz == 1) {
            # print('kbmf1k1k')
            train_function <- kbmf1k1k_semisupervised_regression_variational_train
            test_function <- kbmf1k1k_semisupervised_regression_variational_test
        }
        if (Px > 1 && Pz == 1) {
            train_function <- kbmf1mkl1k_semisupervised_regression_variational_train
            test_function <- kbmf1mkl1k_semisupervised_regression_variational_test
        }
        if (Px == 1 && Pz > 1) {
            train_function <- kbmf1k1mkl_semisupervised_regression_variational_train
            test_function <- kbmf1k1mkl_semisupervised_regression_variational_test
        }
        if (Px > 1 && Pz > 1) {
            train_function <- kbmf1mkl1mkl_semisupervised_regression_variational_train
            test_function <- kbmf1mkl1mkl_semisupervised_regression_variational_test
        }
    }
    if (missing(varargin) == FALSE) {
        for (name in names(varargin)) {
            parameters[[name]] <- varargin[[name]]
        }
    }
    parameters$train_function <- train_function
    parameters$test_function <- test_function
    
    state <- train_function(drop(Kx), drop(Kz), Y, parameters)
}

In [ ]:
kbmf_regression_test <- function(Kx, Kz, state) {
    # ====================================================
    # Fonction de test du modèle
    # Paramètres en entrée  
    # Kx
    # Kz
    # state
    # 
    # 
    # 
    # 
    # Paramètres en sortie
    # 
    # 
    #
    # ====================================================
    prediction <- state$parameters$test_function(drop(Kx), drop(Kz), state)
    }

In [ ]:
# Entrainement
state <- kbmf_regression_train(Kx, Kz, Y, 5, iter = 20) # Le R est le rang??

In [ ]:
# Test du modèle
prediction <- kbmf_regression_test(Kx, Kz, state)

# Présentation des résultats

In [ ]:
print(sprintf("RMSE = %.4f", sqrt(mean((prediction$Y$mu - Y) ^ 2))))

In [ ]:
print("kernel weights on X")
print(state$ex$mu)

In [ ]:
print("kernel weights on Z")
print(state$ez$mu)